In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from src.local.configs import BenchmarksConfig
from src.local.manager import BenchmarksManager

In [0]:
cfg = BenchmarksConfig.load_from_json("../../benchmarks_config/databricks/small.json")
manager = BenchmarksManager(configs=cfg)
result = manager.run()
# print(result)
